[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ryanfobel/gridwatch-history/main)

# Calculate CO2e intensity for the Ontario grid

Try to calculate the grid intensity from publically available data to see if we can match the data provided by gridwatch. See [issue #1](https://github.com/ryanfobel/gridwatch-history/issues/1) on github.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.insert(0, os.path.join(".."))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams

from gridwatch_scraper import load_file

%matplotlib inline

rcParams.update({"figure.figsize": (12, 6)})

In [2]:
# Download IESO historical data
import requests


def download_url(url, ext='.xlsx'):
    filename = os.path.join('..', 'data', 'raw', 'ieso.ca', os.path.splitext(url.split('/')[-1])[0] + ext)
    if not os.path.exists(filename):
        print(f"Download { filename }")
        r = requests.get(url)
        if r.ok:
            with open(filename,'wb') as output_file:
                output_file.write(r.content)
        else:
            raise RuntimeError("Error downloading file")


def cleanup(df):
    df = df[pd.notna(df["DATE"])]
    df["HOUR"] = df["HOUR"] - 1
    df.index = pd.to_datetime([f'{row["DATE"].date().isoformat()} {int(row["HOUR"]):02}:00:00' for index, row in df.iterrows()])
    return df.drop(columns=["DATE", "HOUR"])


years = range(2010, 2020)
df = pd.DataFrame(
    {
        "url": [f"https://ieso.ca/-/media/Files/IESO/Power-Data/data-directory/GOC-{ year }.ashx" for year in years]
    },
    index=years
)
df.loc[2019, "url"] = 'https://ieso.ca/-/media/Files/IESO/Power-Data/data-directory/GOC-2019-Jan-April.ashx'
df["ext"] = ".xlsx"
df

,url,ext
2010,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2011,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2012,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2013,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2014,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2015,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2016,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2017,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2018,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx
2019,https://ieso.ca/-/media/Files/IESO/Power-Data/...,.xlsx


In [3]:
for year, row in df.iterrows():
    download_url(row["url"])

    output_path = os.path.join("..", "data", "clean", "ieso.ca", "output", f"{year}.csv")
    if not os.path.exists(output_path):
        print(year)
        url = row["url"]
        filename = os.path.join('..', 'data', 'raw', 'ieso.ca', os.path.splitext(url.split('/')[-1])[0] + row["ext"])
        df = pd.read_excel(filename, engine='openpyxl')
        drop_columns = {
            2010: "Unnamed: 2",
            2011: "Unnamed: 2",
            2012: "a",
        }
        if year in drop_columns.keys():
            df = df.drop(columns=[drop_columns[year]])
        if "Hour" in df.columns:
            df = df.rename(columns={"Hour": "HOUR"})
        if "Date" in df.columns:
            df = df.rename(columns={"Date": "DATE"})

        df = cleanup(df)

        # Add TOTAL column if it doesn't exist
        if "TOTAL" not in df.columns:
            df["TOTAL"] = df.sum(axis=1)
        # Put TOTAL first
        df = df[(["TOTAL"] + [col for col in df.columns if col != "TOTAL"])]

        df.to_csv(output_path)

In [4]:
# Rest of 2019 onwards
year = 2019
month = 6

while True:
    try:
        url = "http://reports.ieso.ca/public/GenOutputCapabilityMonth/PUB_GenOutputCapabilityMonth_%d%02d.csv" % (year, month)
        download_url(url, ext='.csv')
        if month == 12:
            year += 1
            month = 1
        else:
            month +=1
    except RuntimeError:
        break

Download ..\data\raw\ieso.ca\PUB_GenOutputCapabilityMonth_202305.csv


In [5]:
from glob import glob
df = pd.DataFrame({
    "filepath": glob(os.path.join("..", "data", "raw", "ieso.ca", "PUB_GenOutputCapabilityMonth_*"))
})
df["filename"] = [os.path.basename(fn) for fn in df["filepath"]]
df = pd.concat([
    df,
    df["filename"].str.extract(r'PUB_GenOutputCapabilityMonth_(?P<year>\d{4})(?P<month>\d{2})')
], axis=1)
df["url"] = [
    'http://reports.ieso.ca/public/GenOutputCapabilityMonth/PUB_GenOutputCapabilityMonth_{}{}.csv'.format(
        row["year"], row["month"]
    ) for index, row in df.iterrows()
]
df

,filepath,filename,year,month,url
0,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201905.csv,2019,05,http://reports.ieso.ca/public/GenOutputCapabil...
1,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201906.csv,2019,06,http://reports.ieso.ca/public/GenOutputCapabil...
2,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201907.csv,2019,07,http://reports.ieso.ca/public/GenOutputCapabil...
3,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201908.csv,2019,08,http://reports.ieso.ca/public/GenOutputCapabil...
4,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201909.csv,2019,09,http://reports.ieso.ca/public/GenOutputCapabil...
5,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201910.csv,2019,10,http://reports.ieso.ca/public/GenOutputCapabil...
6,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201911.csv,2019,11,http://reports.ieso.ca/public/GenOutputCapabil...
7,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_201912.csv,2019,12,http://reports.ieso.ca/public/GenOutputCapabil...
8,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_202001.csv,2020,01,http://reports.ieso.ca/public/GenOutputCapabil...
9,..\data\raw\ieso.ca\PUB_GenOutputCapabilityMon...,PUB_GenOutputCapabilityMonth_202002.csv,2020,02,http://reports.ieso.ca/public/GenOutputCapabil...


In [6]:
def cleanup_monthly_data(df_input: pd.DataFrame, measurement: str="Output") -> pd.DataFrame:
    df = pd.DataFrame()
    output_mask = df_input["Measurement"] == measurement
    columns = ["Generator"] + [f"Hour {x}" for x in range(1,25)]
    for date in df_input["Delivery Date"].unique():
        index = df_input[(df_input["Delivery Date"] == date) & output_mask].index
        df_output = df_input.loc[index, columns].set_index("Generator").T
        df_output.index = pd.to_datetime([f"{date}T{x:02}:00:00" for x in range(24)])
        df = pd.concat([
            df,
            df_output
        ], axis=0)
    return df


yearly_data = {}
for index, row in df.iterrows():
    print(row["year"], row["month"])
    if row["year"] not in yearly_data.keys():
        yearly_data[year] = pd.DataFrame()
    
    yearly_data[year] = pd.concat([
        yearly_data[year],
        cleanup_monthly_data(pd.read_csv(row["filepath"], skiprows=3, index_col=False))
    ], axis=0)
yearly_data

2019 05
2019 06
2019 07
2019 08
2019 09
2019 10
2019 11
2019 12
2020 01
2020 02
2020 03
2020 04
2020 05
2020 06
2020 07
2020 08
2020 09
2020 10
2020 11
2020 12
2021 01
2021 02
2021 03
2021 04
2021 05
2021 06
2021 07
2021 08
2021 09
2021 10
2021 11
2021 12
2022 01
2022 02
2022 03
2022 04
2022 05
2022 06
2022 07
2022 08
2022 09
2022 10
2022 11
2022 12
2023 01
2023 02
2023 03
2023 04


{2023: Generator           ABKENORA ADELAIDE AGUASABON ALEXANDER AMARANTH  \
 2023-04-01 00:00:00       15       44        19        49       43   
 2023-04-01 01:00:00       15       47        19        49       69   
 2023-04-01 02:00:00       15       52        19        49       80   
 2023-04-01 03:00:00       14       52        19        49       95   
 2023-04-01 04:00:00       14       59        19        49       70   
 ...                      ...      ...       ...       ...      ...   
 2023-04-21 19:00:00       15        3        43         3        2   
 2023-04-21 20:00:00       15       10        42         3        4   
 2023-04-21 21:00:00       15       22        42         3        2   
 2023-04-21 22:00:00       15        7        43         3        2   
 2023-04-21 23:00:00       15        1        42         3        9   
 
 Generator           AMHERST ISLAND APIROQUOIS ARMOW ARNPRIOR ATIKOKAN-G1  ...  \
 2023-04-01 00:00:00             11         57   122      

In [7]:
from CA import fetch_production
import arrow

now = arrow.now()

In [8]:
# 3-months of historical data available via IESO xml feed [1]
# 1. https://github.com/electricitymaps/electricitymaps-contrib/blob/master/parsers/CA_ON.py)
data = fetch_production(target_datetime=now.shift(months=-3).datetime)
print(len(data))
data

24


[{'datetime': datetime.datetime(2023, 1, 24, 1, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400), 'UTC-5')),
  'zoneKey': 'CA-ON',
  'production': {'biomass': 24.0,
   'gas': 131.0,
   'hydro': 3581.0,
   'nuclear': 10048.0,
   'solar': 0.0,
   'wind': 3895.0},
  'storage': {},
  'source': 'ieso.ca'},
 {'datetime': datetime.datetime(2023, 1, 24, 2, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400), 'UTC-5')),
  'zoneKey': 'CA-ON',
  'production': {'biomass': 23.0,
   'gas': 132.0,
   'hydro': 3618.0,
   'nuclear': 10056.0,
   'solar': 0.0,
   'wind': 3919.0},
  'storage': {},
  'source': 'ieso.ca'},
 {'datetime': datetime.datetime(2023, 1, 24, 3, 0, tzinfo=datetime.timezone(datetime.timedelta(-1, 68400), 'UTC-5')),
  'zoneKey': 'CA-ON',
  'production': {'biomass': 23.0,
   'gas': 132.0,
   'hydro': 3753.0,
   'nuclear': 10059.0,
   'solar': 0.0,
   'wind': 4086.0},
  'storage': {},
  'source': 'ieso.ca'},
 {'datetime': datetime.datetime(2023, 1, 24, 4, 0, tzinfo=dateti

In [9]:
df = pd.json_normalize(fetch_production(target_datetime=now))
print(now)
df

2023-04-24T14:33:25.110426-04:00


,datetime,zoneKey,source,production.biomass,production.gas,production.hydro,production.nuclear,production.solar,production.wind
0,2023-04-24 01:00:00-05:00,CA-ON,ieso.ca,0.0,532.0,5168.0,8717.0,0.0,289.0
1,2023-04-24 02:00:00-05:00,CA-ON,ieso.ca,0.0,503.0,4919.0,8719.0,0.0,404.0
2,2023-04-24 03:00:00-05:00,CA-ON,ieso.ca,0.0,567.0,5030.0,8719.0,0.0,398.0
3,2023-04-24 04:00:00-05:00,CA-ON,ieso.ca,0.0,842.0,4702.0,8711.0,0.0,460.0
4,2023-04-24 05:00:00-05:00,CA-ON,ieso.ca,0.0,1327.0,5122.0,8714.0,0.0,429.0
5,2023-04-24 06:00:00-05:00,CA-ON,ieso.ca,0.0,1645.0,5713.0,8714.0,0.0,359.0
6,2023-04-24 07:00:00-05:00,CA-ON,ieso.ca,0.0,1885.0,6136.0,8715.0,23.0,320.0
7,2023-04-24 08:00:00-05:00,CA-ON,ieso.ca,0.0,1817.0,6122.0,8715.0,70.0,292.0
8,2023-04-24 09:00:00-05:00,CA-ON,ieso.ca,0.0,1681.0,5989.0,8712.0,139.0,341.0
9,2023-04-24 10:00:00-05:00,CA-ON,ieso.ca,0.0,1654.0,5853.0,8711.0,164.0,375.0


![biomass](images/co2signal/biomass.png)
![coal](images/co2signal/coal.png)
![gas](images/co2signal/gas.png)
![geothermal](images/co2signal/geothermal.png)
![hydro](images/co2signal/hydro.png)
![nuclear](images/co2signal/nuclear.png)
![oil](images/co2signal/oil.png)
![solar](images/co2signal/solar.png)
![wind](images/co2signal/wind.png)

In [10]:
df_intensity = pd.DataFrame({
    "biomass": {
        "carbon_intensity": 230,
        "source": "IPCC 2014"
    },
    "coal": {
        "carbon_intensity": 820,
        "source": "IPCC 2014"
    },
    "gas": {
        "carbon_intensity": 490,
        "source": "IPCC 2014"
    },
    "geothermal": {
        "carbon_intensity": 38,
        "source": "IPCC 2014"
    },
    "hydro": {
        "carbon_intensity": 24,
        "source": "IPCC 2014"
    },
    "nuclear": {
        "carbon_intensity": 5,
        "source": "Mallia, E., Lewis, G. 'Life cycle greenhouse gas emissions of electricity generation in the province of Ontario, Canada' (2013)"
    },
    "oil": {
        "carbon_intensity": 650,
        "source": "UK POST 2014"
    },
    "solar": {
        "carbon_intensity": 45,
        "source": "IPCC 2014"
    },
    "wind": {
        "carbon_intensity": 11,
        "source": "IPCC 2014"
    },
})
df_intensity

,biomass,coal,gas,geothermal,hydro,nuclear,oil,solar,wind
carbon_intensity,230,820,490,38,24,5,650,45,11
source,IPCC 2014,IPCC 2014,IPCC 2014,IPCC 2014,IPCC 2014,"Mallia, E., Lewis, G. 'Life cycle greenhouse g...",UK POST 2014,IPCC 2014,IPCC 2014


In [11]:
sources = [col[len("production."):] for col in df.columns if col.startswith("production.")]

df["production.total"] = 0
for source in sources:
    df["production.total"] = df["production.total"] + df[f"production.{source}"]

df["emissions.total"] = 0
for source in sources:
    df[f"emissions.{source}"] = df[f"production.{source}"] * df_intensity[source].carbon_intensity
    df["emissions.total"] = df["emissions.total"] + df[f"emissions.{source}"]

In [12]:
df["carbon_intensity"] = df["emissions.total"] / df["production.total"] # kg/MWh or g/kWh

# production in MW, emissions in kg CO2e, carbon intensity (gCO2e/kWh)
df[["production.total", "emissions.total", "carbon_intensity"]]

,production.total,emissions.total,carbon_intensity
0,14706.0,431476.0,29.340133
1,14545.0,412565.0,28.364730
2,14714.0,446523.0,30.346813
3,14715.0,574043.0,39.010737
4,15592.0,821447.0,52.683876
5,16431.0,990681.0,60.293409
6,17079.0,1119044.0,65.521635
7,17016.0,1087195.0,63.892513
8,16862.0,1020992.0,60.549875
9,16757.0,1005992.0,60.034135


In [13]:
import os
import json
from dotenv import load_dotenv

load_dotenv()

country_code = "CA-ON"
url = f"https://api.co2signal.com/v1/latest?countryCode={country_code}\""
response = requests.get(url, headers={'auth-token': f'{os.environ["CO2SIGNAL_API_TOKEN"]}'})
if response.status_code != 200:
    raise RuntimeError(response.status_code)
json.loads(response.content)

{'_disclaimer': "This data is the exclusive property of Electricity Maps and/or related parties. If you're in doubt about your rights to use this data, please contact api@co2signal.com",
 'status': 'ok',
 'countryCode': 'CA-ON"',
 'data': {'fossilFuelPercentage': None},
 'units': {'carbonIntensity': 'gCO2eq/kWh'}}